In [1]:
1) Create term cluster for each of 4 weeks
3) Create weekly / daily stats for each term 
https://docs.databricks.com/notebooks/notebooks-use.html

File "<command-807861565787517>" , line 1 
 1) Create term cluster for each of 4 weeks 
 ^
 SyntaxError : invalid syntax

In [2]:
import sparknlp

print("Spark NLP version")
print(sparknlp.version())
print("Apache Spark version")

Spark NLP version
2.5.1
Apache Spark version

In [3]:
!pip install wordcloud

In [4]:
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.functions import from_unixtime, to_date, asc, year, udf, explode, split, col, desc, length, rank, dense_rank, avg, sum
from pyspark.sql.window import Window
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.stat import Correlation
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import col, to_timestamp,date_format
from collections import Counter
from wordcloud import WordCloud

#from textblob import TextBlob

In [5]:
df = spark.sql("""select event_date, full_text as text from january_covid_tweets""")


In [6]:
data = df.withColumn("week", date_format(col("event_date"), "W"))

In [7]:
data.orderBy(asc("event_date")).select("text").show(2)

+--------------------+
 text|
+--------------------+
RT @CDCgov: While...|
RT @CDCgov: The f...|
+--------------------+
only showing top 2 rows

In [8]:
documentExplainerPipeline = PretrainedPipeline('explain_document_dl', 'en')

explain_document_dl download started this may take some time.
Approx size to download 168.4 MB
[ | ][OK!]

In [9]:
week4DF = data.filter((data['week'] == 4)).select("text")

In [10]:
week5DF = data.filter((data['week'] == 5)).select("text")

In [11]:
week4DF.show(2)

+--------------------+
 text|
+--------------------+
RT @StephenMcDone...|
RT @BillRatchet: ...|
+--------------------+
only showing top 2 rows

In [12]:
!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
eng_stopwords = stopwords.words('english')
#eng_stopwords.append('xxxx')
#for item in eng_stopwords[:10]:
#  print(item)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!

In [13]:
def wordcloud(corpus_sdf):
    corpus_pdf = corpus_sdf.limit(500).toPandas()

    corpus_dict = {}
    for index, row in corpus_pdf.iterrows():
        corpus_dict[row['text']] = row['count']
        
    wordcloud = WordCloud().generate_from_frequencies(corpus_dict)
    plt.imshow(wordcloud);

### Approach 1

In [15]:
def remove_stopwords(x):    
    sw = stopwords.words("english")
    string = ''
    for x in x.split(' '):
        if x.lower() not in sw:
            string += x + ' '
        else:
            pass
    return string

nosw = udf(remove_stopwords)
spark.udf.register("nosw", nosw)
week4DF = week4DF.withColumn('text_nosw',nosw('text'))

In [16]:
def corpus_creator(text_col):
    corpus = text_col.rdd \
                    .flatMap(flat_list) \
                    .map(lambda x: (x, 1)) \
                    .reduceByKey(lambda x, y: x+y ) \
                    .sortBy(lambda x: x[1], ascending=False) \
                    .toDF() \
                    .withColumnRenamed('_1','text') \
                    .withColumnRenamed('_2','count')
    return corpus

In [17]:
def flat_list(column):
    corpus = []
    for row in column:
        for w in row.split(' '):
            corpus.append(w)
    return corpus

In [18]:
corpus_tweet=corpus_creator(week4DF.select("text_nosw"))

In [19]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots()
ax = wordcloud(corpus_tweet)
fig.suptitle('{} wordcloud'.format("2020-Jan-W4"))

In [20]:
week5DF = week5DF.withColumn('text_nosw',nosw('text'))
corpus_tweet_janw5=corpus_creator(week5DF.select("text_nosw"))
fig, ax = plt.subplots()
ax = wordcloud(corpus_tweet_janw5)
fig.suptitle('{} wordcloud'.format("2020-Jan-W5"))

### Approach 2

In [22]:
import matplotlib.pyplot as plt
import seaborn as sns

def make_string(x):
  string = ''
  for x in x:
    string += x + ' '
  return string

make_string = udf(make_string)
spark.udf.register("make_string", make_string)   

In [23]:
def resolveEntities(input_corpus, week_list):
  weekly_entities = {}
  print(week_list)
  input_corpus.show(5)
  
  for weeknum in week_list:
      print("starting entity resolution")
      entities_filtered = documentExplainerPipeline.transform(input_corpus) \
                                  .select('text','count',
                                          col('entities.result').alias('entities'),
                                          col('pos.result').alias('pos'))

      entities_filtered.show(2)

      entities_filtered = entities_filtered.withColumn('entities',make_string('entities'))\
                                          .withColumn('pos',make_string('pos'))\
                                          .filter('entities <> ""')

      entities_filtered.show(2)
      weekly_entities[str(weeknum)] = entities_filtered
  return weekly_entities 

In [24]:
week_list = [4]
jan_weekly_entities = resolveEntities(corpus_tweet,week_list)

In [25]:
for key,value in jan_weekly_entities.items():
    fig, ax = plt.subplots()
    ax = wordcloud(value)
    fig.suptitle('{} wordcloud'.format(key))

## Approach 3

In [27]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer,LemmatizerModel, StopWordsCleaner)
from pyspark.ml import Pipeline
documentAssembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# note normalizer defaults to changing all words to lowercase.
# Use .setLowercase(False) to maintain input case.
normalizer = Normalizer() \
    .setInputCols(['token']) \
    .setOutputCol('normalized') \
    .setLowercase(True)

# note that lemmatizer needs a dictionary. So I used the pre-trained
# model (note that it defaults to english)
lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(['normalized']) \
    .setOutputCol('lemma') \

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(['lemma']) \
    .setOutputCol('clean_lemma') \
    .setCaseSensitive(False) \
    .setStopWords(eng_stopwords) 

#eng_stopwords

# finisher converts tokens to human-readable output
finisher = Finisher() \
    .setInputCols(['clean_lemma']) \
    .setCleanAnnotations(False)

pipeline = Pipeline() \
    .setStages([
        documentAssembler,
        tokenizer,
        normalizer,
        lemmatizer,
        stopwords_cleaner,
        finisher
    ])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [28]:
week4CleanDF = pipeline.fit(week4DF).transform(week4DF) # without stop words

In [29]:
week5CleanDF = pipeline.fit(week5DF).transform(week5DF)

In [30]:
week4CleanDFV2 = pipeline.fit(week4DF).transform(week4DF)
week4CleanDFV2.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| text_nosw| document| token| normalized| lemma| clean_lemma|finished_clean_lemma|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
RT @StephenMcDone...|RT @StephenMcDone...|[[document, 0, 13...|[[token, 0, 1, RT...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[rt, stephenmcdon...|
RT @BillRatchet: ...|RT @BillRatchet: ...|[[document, 0, 89...|[[token, 0, 1, RT...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[rt, billratchet,...|
RT @haruharu_w_bt...|RT @haruharu_w_bt...|[[document, 0, 14...|[[token, 0, 1, RT...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[rt, haruharuwbts...|
RT @haruharu_w_bt...|RT @haruharu_w_bt...|[[document, 0, 14...|[[token, 0, 1, RT...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[rt, haruharuwbts...|
RT @botchedcredit...|RT @botchedcredit...|[[document, 0, 11...|[[token, 0, 1, RT...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[rt, botchedcredi...|
RT @AFP: #UPDATE ...|RT @AFP: #UPDATE ...|[[document, 0, 13...|[[token, 0, 1, RT...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[rt, afp, update,...|
RT @celliottabili...|RT @celliottabili...|[[document, 0, 13...|[[token, 0, 1, RT...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[rt, celliottabil...|
RT @TrnThiHa14: W...|RT @TrnThiHa14: W...|[[document, 0, 71...|[[token, 0, 1, RT...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[rt, trnthiha, wu...|
RT @TrnThiHa14: W...|RT @TrnThiHa14: W...|[[document, 0, 71...|[[token, 0, 1, RT...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[rt, trnthiha, wu...|
RT @Miedle3: ++Br...|RT @Miedle3: ++Br...|[[document, 0, 13...|[[token, 0, 1, RT...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[[token, 0, 1, rt...|[rt, miedle, ++br...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 10 rows

In [31]:
from pyspark.sql.functions import explode, col
week4TokenDF = week4CleanDF.withColumn("exploded_text", explode(col("finished_clean_lemma")))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3320953040114910> in <module> 
 1 from pyspark . sql . functions import explode , col
 2 week4TokenDF = week4CleanDF . withColumn ( "exploded_text" , explode ( col ( "finished_clean_lemma" ) ) ) 
 ----> 3 week5TokenDF = week5CleanDF . withColumn ( "exploded_text" , explode ( col ( "finished_clean_lemma" ) ) ) 

 NameError : name 'week5CleanDF' is not defined

In [32]:
week5TokenDF = week5CleanDF.withColumn("exploded_text", explode(col("finished_clean_lemma")))

In [33]:
countWeek4DF = week4TokenDF.groupby('exploded_text').count()
countWeek4DF = countWeek4DF.withColumnRenamed("exploded_text","text")
#countWeek4DF.show()

In [34]:
countWeek5DF = week5TokenDF.groupby('exploded_text').count()
countWeek5DF = countWeek5DF.withColumnRenamed("exploded_text","text")
#countWeek4DF.show()

In [35]:
countWeek5DF.repartition(150).write.format("orc").mode('overwrite').saveAsTable("countJanWeek5DF")

In [36]:
topTermsWeek4DF = spark.sql("""select text, count from countJanWeek4DF where count > 1000 order by count desc limit 100""")
#%sql
#select text, count from countJanWeek4DF where count > 1000 order by count desc limit 50

In [37]:
topTermsWeek5DF = spark.sql("""select text, count from countJanWeek5DF where count > 1000 order by count desc limit 100""")
#topTermsDF.show(100)


--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 

 Py4JJavaError : An error occurred while calling o214.sql.
: org.apache.spark.sql.AnalysisException: Table or view not found: countJanWeek5DF; line 1 pos 24
	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:47)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$.org$apache$spark$sql$catalyst$analysis$Analyzer$ResolveRelations$$lookupTableFromCatalog(Analyzer.scala:755)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$.resolveRelation(Analyzer.scala:699)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$$anonfun$apply$8.applyOrElse(Analyzer.scala:735)
	at org.apache.spark.sql.catalyst.analysis.Analyzer$ResolveRelations$$anonfun$apply$8.applyOrElse(Analyzer.scala:728)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$apply$1.apply(AnalysisHelper.scala:90)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$apply$1.apply(AnalysisHelper.scala:90)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(TreeNode.scala:76)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:89)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsUp(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUp(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.allowInvokingTransformsInAnalyzer(AnalysisHelper.scala:194)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$class.resolveOperatorsUp(AnalysisHelper.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveOperatorsUp(LogicalPlan.scala:29)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1$$anonfun$1.apply(AnalysisHelper.scala:87)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$8.apply(TreeNode.scala:353)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapProductIterator(TreeNode.scala:207)
	at org.apache.spark.sql.catalyst.trees.TreeNode.mapChildren(TreeNode.scala:351)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$$anonfun$resolveOperatorsUp$1.apply(

In [38]:
fig, ax = plt.subplots()
ax = wordcloud(topTermsWeek4DF)
fig.suptitle('{} wordcloud'.format("2020-01-W4"))

In [39]:
topTermsWeek4DF.show(2)
jan_weekly_entities['4'].show()

+-----------+-------+
 text| count|
+-----------+-------+
 rt|1342192|
coronavirus| 826204|
+-----------+-------+
only showing top 2 rows

+-----------------+-------+------------------+------+
 text| count| entities| pos|
+-----------------+-------+------------------+------+
 RT|1273166| RT | NNP |
 Wuhan| 239418| Wuhan | NNP |
 China| 223525| China | NNP |
 Chinese| 107526| Chinese | JJ |
 Coronavirus| 78013| Coronavirus | NN |
 #Wuhan| 73936| #Wuhan | NNP |
 &amp;| 56330| &amp | NN : |
 Hong| 50826| Hong | NNP |
 Wuhan,| 46795| Wuhan |NNP , |
 US| 44092| US | NNP |
#WuhanCoronavirus| 33194|#WuhanCoronavirus | NN |
 #China| 31872| #China | NNP |
 CDC| 30479| CDC | NNP |
 Kong| 29924| Kong | NNP |
 #Coronavirus| 27273| #Coronavirus | NN |
 New| 24505| New | NNP |
 United| 24410| United | NNP |
 U.S.| 23918| U.S |NNP . |
 States| 21810| States | NNS |
 University| 20052| University | NNP |
+-----------------+-------+------------------+------+
only showing top 20 rows

In [40]:
fig, ax = plt.subplots()
ax = wordcloud(topTermsWeek5DF)
fig.suptitle('{} wordcloud'.format("2020-01-W5"))

java.rmi.RemoteException: com.databricks.api.base.DatabricksServiceException: INTERNAL_ERROR: Unexpected error.; nested exception is: 
	com.databricks.api.base.DatabricksServiceException: INTERNAL_ERROR: Unexpected error.
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:123)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIgnoreDraining(DbfsClient.scala:83)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close0(DbfsOutputStream.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsOutputStream.close(DbfsOutputStream.scala:79)
	at java.io.FilterOutputStream.close(FilterOutputStream.java:159)
	at org.apache.hadoop.fs.FSDataOutputStream$PositionCache.close(FSDataOutputStream.java:72)
	at org.apache.hadoop.fs.FSDataOutputStream.close(FSDataOutputStream.java:106)
	at org.apache.hadoop.io.IOUtils.copyBytes(IOUtils.java:61)
	at org.apache.hadoop.io.IOUtils.copyBytes(IOUtils.java:105)
	at org.apache.hadoop.fs.FileUtil.copy(FileUtil.java:443)
	at com.databricks.backend.daemon.driver.FileStorePlotImageSaver.savePlotImage(FileStorePlotImageSaver.scala:32)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$getResultBufferInternal$1$$anonfun$11.apply(PythonDriverLocal.scala:962)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$getResultBufferInternal$1$$anonfun$11.apply(PythonDriverLocal.scala:962)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.TraversableLike$$anonfun$map$1.apply(TraversableLike.scala:234)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1334)
	at scala.collection.IterableLike$class.foreach(IterableLike.scala:72)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:54)
	at scala.collection.TraversableLike$class.map(TraversableLike.scala:234)
	at scala.collection.AbstractTraversable.map(Traversable.scala:104)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$getResultBufferInternal$1.apply(PythonDriverLocal.scala:962)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$getResultBufferInternal$1.apply(PythonDriverLocal.scala:936)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:881)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.getResultBufferInternal(PythonDriverLocal.scala:936)
	at com.databricks.backend.daemon.driver.DriverLocal.getResultBuffer(DriverLocal.scala:492)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.com$databricks$backend$daemon$driver$PythonDriverLocal$$outputSuccess(PythonDriverLocal.scala:923)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$repl$6.apply(PythonDriverLocal.scala:369)
	at com.databricks.backend.daemon.driver.PythonDriverLocal$$anonfun$repl$6.apply(PythonDriverLocal.scala:356)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.withInterpLock(PythonDriverLocal.scala:881)
	at com.databricks.backend.daemon.driver.PythonDriverLocal.repl(PythonDriverLocal.scala:356)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$9.apply(DriverLocal.scala:396)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$9.apply(DriverLocal.scala:373)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging$class.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionContext(DriverLocal.scala:49)
	at com.databricks.logging.UsageLogging$class.withAttributionTags(UsageLogging.scala:275)
	at com.databricks.backend.daemon.driver.DriverLocal.withAttributionTags(DriverLocal.scala:49)
	at com.databricks.backend.daemon.driver.DriverLocal.execute(DriverLocal.scala:373)
	at com.databricks.backend.daemon.driver

##Approach 4

In [42]:
week_list = [4]
jan_weekly_entities_v2 = resolveEntities(topTermsWeek4DF,week_list)

[4]
+-----------+-------+
 text| count|
+-----------+-------+
 rt|1342192|
coronavirus| 826204|
 wuhan| 467435|
 china| 343293|
 case| 186141|
+-----------+-------+
only showing top 5 rows

starting entity resolution
+-----------+-------+--------+-----+
 text| count|entities| pos|
+-----------+-------+--------+-----+
 rt|1342192| []|[NNP]|
coronavirus| 826204| []| [NN]|
+-----------+-------+--------+-----+
only showing top 2 rows

+----+-----+--------+---+
text|count|entities|pos|
+----+-----+--------+---+
 ive| 8376| ive |JJ |
+----+-----+--------+---+

In [43]:

for key,value in jan_weekly_entities_v2.items():
    fig, ax = plt.subplots()
    ax = wordcloud(value)
    fig.suptitle('{} wordcloud'.format(key))

In [44]:
topTermsWeek4DF.show(20)

+-----------+-------+
 text| count|
+-----------+-------+
 rt|1342192|
coronavirus| 826204|
 wuhan| 467435|
 china| 343293|
 case| 186141|
 outbreak| 181430|
 virus| 168359|
 new| 161240|
 people| 131099|
 confirm| 128874|
 chinese| 122035|
 hospital| 115500|
 health| 95157|
 spread| 93908|
 city| 82392|
 break| 81646|
 say| 76658|
 patient| 74939|
 life| 73853|
 first| 71553|
+-----------+-------+
only showing top 20 rows

In [45]:
from sparknlp.pretrained import PretrainedPipeline
pipeline1 = PretrainedPipeline('recognize_entities_dl', 'en')

pipeline2 = PretrainedPipeline("onto_recognize_entities_sm", "en")

predictions1 = pipeline1.transform(data)
predictions1EntityResult=predictions1.select("entities.result")
predictions1EntityResult.show(10)
#predictions1EntityResult.printSchema()

predictions1NerResult=predictions1.select("ner.result")
predictions1NerResult.show(1)
predictions1NerResult.printSchema()

predictions2 = pipeline2.transform(data)
predictions2.select("entities.result").show(5)
predictions2.select("ner.result").show(2)

+--------------------+
 result|
+--------------------+
[B-ORG, I-ORG, O,...|
+--------------------+
only showing top 1 row

root
-- result: array (nullable = true)
 |-- element: string (containsNull = true)